In [483]:
import pandas as pd
import numpy as np

In [484]:
from datetime import datetime

target_date = datetime(2021, 4, 16)
target_date_string = target_date.strftime("%Y-%m-%d")
print(target_date_string)

requires_two_days = False

2021-04-16


In [485]:
csv_test=pd.read_csv('labeled_tweet_data_{}.csv'.format(target_date_string))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_46832\3363056945.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_test=pd.read_csv('C:\\Users\\j7206\\OneDrive - SNU\\문서\\카카오톡 받은 파일\\감정분석_추가\\labeled_tweet_data_{}.csv'.format(target_date_string))


In [486]:
csv_test=csv_test.sort_values(by='created_at')
csv_test=csv_test.reset_index(inplace=False)

In [487]:
csv_test=csv_test[['created_at','likes_count','sentiment']]

In [488]:
csv_test=csv_test.replace('positive', 1)
csv_test=csv_test.replace('neutral', 0)
csv_test=csv_test.replace('negative', -1)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_46832\3428117729.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  csv_test=csv_test.replace('negative', 0)


In [489]:
csv_test.insert(3,'average',0)
csv_test.insert(4,'weighted_average',0)
csv_test.insert(5,'n_positive',0)
csv_test.insert(6,'n_neutral',0)
csv_test.insert(7,'n_negative',0)
csv_test.insert(8,'n_total',0)

In [490]:
test=np.zeros(1440 if not requires_two_days else 2880)
sample_df=pd.DataFrame({'time':test})

In [491]:
# 가능한 모든 분 채우기
def convert(origin: str) -> datetime:
    plus_index = origin.rfind("+")
    sliced = origin[:plus_index]

    start_datetime = datetime.strptime(sliced, "%Y-%m-%d %H:%M:%S")
    start_datetime = start_datetime.replace(second=0)
    return start_datetime

start_time = csv_test["created_at"].iloc[1]
start_datetime = convert(start_time)
print(start_datetime)

2021-04-16 11:00:00


In [492]:
from datetime import timedelta

target_datetime = start_datetime

for n in range(1440 if not requires_two_days else 2880):
    sample_df["time"][n] = target_datetime.strftime("%Y-%m-%dT%H:%M:%S")
    target_datetime += timedelta(minutes=1)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_46832\1623163407.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample_df["time"][n] = target_datetime.strftime("%Y-%m-%dT%H:%M:%S")
C:\Users\Public\Documents\ESTsoft\CreatorTemp\i

In [493]:
sample_df

,time
0,2021-04-16T11:00:00
1,2021-04-16T11:01:00
2,2021-04-16T11:02:00
3,2021-04-16T11:03:00
4,2021-04-16T11:04:00
...,...
1435,2021-04-17T10:55:00
1436,2021-04-17T10:56:00
1437,2021-04-17T10:57:00
1438,2021-04-17T10:58:00


In [494]:
# csv_test_sp 도 바꿔야됨
def convert_format(origin: str) -> str:
    dt = convert(origin)
    return dt.strftime("%Y-%m-%dT%H:%M:%S")

csv_test["created_at"] = csv_test["created_at"].apply(lambda x: convert_format(x))

csv_test.head()

,created_at,likes_count,sentiment,average,weighted_average,n_positive,n_neutral,n_negative,n_total
0,2021-04-16T11:00:00,0.0,1,0,0,0,0,0,0
1,2021-04-16T11:00:00,0.0,-1,0,0,0,0,0,0
2,2021-04-16T11:00:00,4.0,-1,0,0,0,0,0,0
3,2021-04-16T11:00:00,1.0,1,0,0,0,0,0,0
4,2021-04-16T11:00:00,2.0,0,0,0,0,0,0,0


In [495]:
sample_df
sample_df.insert(1,'average',0)
sample_df.insert(2,'weighted_average',0)
sample_df.insert(3,'n_positive',0)
sample_df.insert(4,'n_neutral',0)
sample_df.insert(5,'n_negative',0)
sample_df.insert(6,'n_total',0)
sample_df.insert(7,'bullishness',0) #추가

In [496]:
start=0
'''avg=0
w_avg=0
n_pos=0
n_neu=0
n_neg=0
n_w=0
n_tot=0'''
for i in range(1440 if not requires_two_days else 2880):
    avg=0
    w_avg=0
    n_pos=0
    n_neu=0
    n_neg=0
    n_w=0
    n_tot=0

    if start == len(csv_test):
        #print(f"out of data! -- at {start = }, {i = }")
        break

    for j in range(start, len(csv_test)):
        if csv_test['created_at'][j] == sample_df['time'][i]:
            #print(f"IF {j = } {csv_test['created_at'][j]} == {sample_df['time'][i]}")
            n_tot+=1
            n_w+=csv_test['likes_count'][j]
            if csv_test['sentiment'][j]==1:
                n_pos+=1
            elif csv_test['sentiment'][j]==0:
                n_neu+=1
            else:
                n_neg+=1
            avg+=csv_test['sentiment'][j]
            w_avg+=csv_test['sentiment'][j]*csv_test['likes_count'][j]
            start+=1
        else:
            break
    if n_tot!=0:
        sample_df['average'][i]=avg/n_tot
        sample_df['weighted_average'][i]=w_avg/n_w
        sample_df['n_positive'][i]=n_pos
        sample_df['n_neutral'][i]=n_neu
        sample_df['n_negative'][i]=n_neg
        sample_df['n_total'][i]=n_tot
        sample_df['bullishness'][i]=np.log((1+n_pos)/(1+n_neg))
    else:
        sample_df['average'][i]=sample_df['average'][i-1]
        sample_df['weighted_average'][i]=sample_df['weighted_average'][i-1]
        sample_df['n_positive'][i]=0
        sample_df['n_neutral'][i]=0
        sample_df['n_negative'][i]=0
        sample_df['n_total'][i]=0
        sample_df['bullishness'][i]=sample_df['bullishness'][i-1]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_46832\60802197.py:39: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample_df['average'][i]=avg/n_tot
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_46832\60802197.py:39: Setti

In [497]:
sample_df.to_csv('sentiment_{}.csv'.format(target_date_string))